## Решение задачи по поиску ответа на основе корпуса вопросов-отвтев

Здесь я использую только word2vec модель для ознакомления работы с этим монстром.

In [1]:
import pickle
import gensim
import string
import warnings
import re
import json
import numpy as np 
import pandas as pd

from pymystem3 import Mystem
from gensim import matutils
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec, KeyedVectors
from collections import defaultdict

warnings.filterwarnings('ignore')
mystem = Mystem()

Импортируем Word2Vec модель и сам корпус.

In [7]:
model_path = '/Users/macbook/Downloads/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'
model = Word2Vec.load(model_path)

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Также я использую стандартную функцию предпроцессинга.

In [4]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

Теперь необходимо написать функцию, которая будет составлять вектор нашего текста, после его подготовки.

In [16]:
def w2v_vector(word_list):
    some = []
    for word in word_list:
        try:
            some.append(model.wv[str(word)])
        except KeyError:
            continue
    return matutils.unitvec(np.array(sum(some) / len(word_list)))

Составляем словарь, который будет хранить в себе вектор вопроса и текстовый ответ.

In [9]:
index, data = 1, defaultdict()

for item in qa_corpus:
    data[index] = [w2v_vector(preprocessing(item[0])), item[1]]
    index += 1

Теперь составим функцию поиска и функцию сравнения векторов

In [10]:
def search(query, result_len):
    query_vector = w2v_vector(preprocessing(query))
    local_data = defaultdict()
    for item in data:
        local_data[similarity(query_vector, data[item][0])] = item 
    for el in sorted(local_data, reverse=True)[:result_len]:
        yield data[local_data[el]][1]

def similarity(v1, v2):
    return np.dot(v1, v2)

Теперь проведем тесты.

Для каждого вопроса в корпусе я составляю его собсвенный вектор и смотрю есть ли соответствующий ответ в топ 5 выдаче поисковика

In [17]:
true = 0

for qa in qa_corpus:
    if qa[1] in list(search(qa[0], 5)):
        true += 1

print('For {} attempts, there are(is) {} correct matches (score:{})'.format(len(qa_corpus), true, true / len(qa_corpus)))

For 1384 attempts, there are(is) 1367 correct matches (score:0.9877167630057804)
